In [2]:
using Distributions, LinearAlgebra, Statistics, StatsFuns
cd("../src/")
include("hcmvn.jl")
include("hchol.jl")
include("generate.jl")

Σ_morton_generate

In [72]:
iters = 5
# ns = [512, 1024, 2048]
ns = [8^2, 16^2]
m = 16
N = 10^5
βs = [0.3, 0.1, 0.03];

In [73]:
# beta1
ans_arr = zeros(length(ns), length(βs), 5, iters)
time_arr = zeros(length(ns), length(βs), 5)
real_arr = zeros(length(ns), length(βs), iters)
h_size_arr = zeros(length(ns), length(βs), iters);

In [74]:
for i in 1:length(ns)
    n = ns[i]
    println(n, " start\n")
    for p in 1:iters
        for j in 1:length(βs)
            Σ_morton = Σ_morton_generate(n, βs[j])
            a = fill(-Inf, n)
            b = rand(Uniform(0, n), n)
            
            B, UV = hchol(Σ_morton, m)
            h_size_arr[i, j, p] += (Base.summarysize(UV) + Base.summarysize(B)) / Base.summarysize(Σ_morton)
        
            time_arr[i, j, 1] += @elapsed ans_arr[i, j, p, 1] += mvn(cholesky(Σ_morton).L, a, b, ns = 10, N = 1000)
            time_arr[i, j, 2] += @elapsed ans_arr[i, j, p, 2] += HMVN(a, b, Σ_morton, m, ns = 10, N = 1000)
            time_arr[i, j, 3] += @elapsed ans_arr[i, j, p, 3] += HCMVN(a, b, Σ_morton, 1, m, ns = 10, N = 1000)
            time_arr[i, j, 4] += @elapsed ans_arr[i, j, p, 4] += HCMVN(a, b, Σ_morton, 2, m, ns = 10, N = 1000)
            time_arr[i, j, 5] += @elapsed ans_arr[i, j, p, 5] += HCMVN(a, b, Σ_morton, 4, m, ns = 10, N = 1000)
            
            real_arr[i, j, p] += mvn(cholesky(Σ_morton).L, a, b, ns = 100, N = 10000)
        end
    end
end

64 start

256 start



InterruptException: InterruptException:

In [70]:
h_size_arr

2×3×5 Array{Float64,3}:
[:, :, 1] =
 0.194038  0.194038  0.172554
 1.19819   0.0       0.0     

[:, :, 2] =
 0.194038  0.194038  0.172554
 0.0       0.0       0.0     

[:, :, 3] =
 0.194038  0.194038  0.172554
 0.0       0.0       0.0     

[:, :, 4] =
 0.194038  0.194038  0.172554
 0.0       0.0       0.0     

[:, :, 5] =
 0.194038  0.194038  0.172554
 0.0       0.0       0.0     

In [71]:
time_arr

2×3×5 Array{Float64,3}:
[:, :, 1] =
  6.78247  6.58139  6.73119
 17.3601   0.0      0.0    

[:, :, 2] =
 178.694  178.398  179.001
 142.088    0.0      0.0  

[:, :, 3] =
 0.0609735  0.0596971  0.0565385
 0.0961091  0.0        0.0      

[:, :, 4] =
 5.21575  5.12462  5.20499
 3.95974  0.0      0.0    

[:, :, 5] =
 9.65338  9.5468  9.52184
 7.69576  0.0     0.0    